<a href="https://colab.research.google.com/github/Levan-Danelia/FRTB/blob/main/FRTB_CMDL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Setup and Initial Data
# Import necessary libraries
import pandas as pd
import numpy as np

# --- Initial Portfolio Data ---
# This data represents the starting positions and their sensitivities.
data = [
    {'Position ID': 1, 'Bucket': 7, 'Instrument': 'silver', 'Maturity': '0M', 'Location': 'none', 'Sensitivity': 77948},
    {'Position ID': 2, 'Bucket': 7, 'Instrument': 'silver', 'Maturity': '0M', 'Location': 'none', 'Sensitivity': -1195917},
    {'Position ID': 3, 'Bucket': 7, 'Instrument': 'silver', 'Maturity': '3M', 'Location': 'none', 'Sensitivity': 857},
    {'Position ID': 4, 'Bucket': 7, 'Instrument': 'silver', 'Maturity': '3M', 'Location': 'none', 'Sensitivity': -13142},
    {'Position ID': 5, 'Bucket': 6, 'Instrument': 'Gas', 'Maturity': '3M', 'Location': 'Netherlands', 'Sensitivity': 121412},
    {'Position ID': 6, 'Bucket': 6, 'Instrument': 'Gas', 'Maturity': '0M', 'Location': 'Netherlands', 'Sensitivity': 196507}
]

# --- Regulatory Parameters ---
# Risk Weights for Commodity Buckets as per Article 325as
risk_weights = {
    6: 0.45,  # Gaseous combustibles
    7: 0.20   # Precious metals
}

# Cross-Bucket Correlation for Commodities as per Article 325au
# Correlation between Bucket 6 (Gaseous combustibles) and Bucket 7 (Precious metals)
gamma_bc = 0.20

# Create an initial DataFrame
portfolio_df = pd.DataFrame(data)

print("--- Initial Setup Complete ---")
print(f"Loaded {len(portfolio_df)} positions.")


--- Initial Setup Complete ---
Loaded 6 positions.


In [ ]:
# Cell 2: Step 1 - Identify Risk Factors and Sensitivities
# In this step, we define the unique risk factor for each position.
# The risk factor is a combination of Instrument, Maturity, and Location.

print("### Step 1: Identify Risk Factors and Sensitivities ###")
print("The initial portfolio with sensitivities for each position:")

# Display the initial portfolio DataFrame
portfolio_df.set_index('Position ID', inplace=True)
display(portfolio_df)

### Step 1: Identify Risk Factors and Sensitivities ###
The initial portfolio with sensitivities for each position:


,Bucket,Instrument,Maturity,Location,Sensitivity
Position ID,,,,,
1,7,silver,0M,none,77948
2,7,silver,0M,none,-1195917
3,7,silver,3M,none,857
4,7,silver,3M,none,-13142
5,6,Gas,3M,Netherlands,121412
6,6,Gas,0M,Netherlands,196507


In [ ]:
# Cell 3: Step 2 - Calculate Net Sensitivities (sk)
# As per Article 325f, paragraph 5, we net sensitivities for identical risk factors.

print("\n### Step 2: Calculate Net Sensitivities (sk) ###")
print("Sensitivities are grouped by their unique risk factor and summed.")

# Define the risk factor components
risk_factor_cols = ['Bucket', 'Instrument', 'Maturity', 'Location']

# Group by risk factors and sum sensitivities to get the net sensitivity
net_sensitivities_df = portfolio_df.groupby(risk_factor_cols).agg(
    Net_Sensitivity_sk=('Sensitivity', 'sum')
).reset_index()

# Create a combined risk factor name for clarity
net_sensitivities_df['Risk_Factor'] = net_sensitivities_df['Instrument'] + '_' + \
                                     net_sensitivities_df['Maturity'] + '_' + \
                                     net_sensitivities_df['Location']

display(net_sensitivities_df[['Bucket', 'Risk_Factor', 'Net_Sensitivity_sk']])


### Step 2: Calculate Net Sensitivities (sk) ###
Sensitivities are grouped by their unique risk factor and summed.


,Bucket,Risk_Factor,Net_Sensitivity_sk
0,6,Gas_0M_Netherlands,196507
1,6,Gas_3M_Netherlands,121412
2,7,silver_0M_none,-1117969
3,7,silver_3M_none,-12285


In [ ]:
# Cell 4: Step 3 - Calculate Weighted Sensitivities (WSk)
# Each net sensitivity is multiplied by its regulatory risk weight (RWk) from Article 325as.

print("\n### Step 3: Calculate Weighted Sensitivities (WSk) ###")
print("Applying regulatory risk weights to the net sensitivities.")

# Map the risk weights to the DataFrame
net_sensitivities_df['Risk_Weight_RWk'] = net_sensitivities_df['Bucket'].map(risk_weights)

# Calculate the weighted sensitivity
net_sensitivities_df['Weighted_Sensitivity_WSk'] = net_sensitivities_df['Net_Sensitivity_sk'] * net_sensitivities_df['Risk_Weight_RWk']

display(net_sensitivities_df[['Bucket', 'Risk_Factor', 'Net_Sensitivity_sk', 'Risk_Weight_RWk', 'Weighted_Sensitivity_WSk']])



### Step 3: Calculate Weighted Sensitivities (WSk) ###
Applying regulatory risk weights to the net sensitivities.


,Bucket,Risk_Factor,Net_Sensitivity_sk,Risk_Weight_RWk,Weighted_Sensitivity_WSk
0,6,Gas_0M_Netherlands,196507,0.45,88428.15
1,6,Gas_3M_Netherlands,121412,0.45,54635.40
2,7,silver_0M_none,-1117969,0.20,-223593.80
3,7,silver_3M_none,-12285,0.20,-2457.00


In [ ]:
# Cell 5: Step 4 & 5 - Intra-Bucket Aggregation (Medium Scenario)
# This function calculates the bucket-specific capital (Kb) by aggregating
# the weighted sensitivities within each bucket using the intra-bucket correlation.

def calculate_kb(bucket_df, scenario='medium'):
    """
    Calculates the bucket-specific capital (Kb) based on Article 325f, paragraph 7.
    """
    # Determine intra-bucket correlation (rho_kl) as per Article 325at
    # In this example, basis and commodity are the same, only tenor differs.
    rho_commodity = 1.0
    rho_tenor = 0.99 # Since maturities are different
    rho_basis = 1.0

    rho_kl_medium = rho_commodity * rho_tenor * rho_basis

    # Adjust correlation based on the scenario as per Article 325h
    if scenario == 'high':
        rho_kl = min(rho_kl_medium * 1.25, 1.0)
    elif scenario == 'low':
        rho_kl = max(2 * rho_kl_medium - 1, 0.75 * rho_kl_medium)
    else: # medium
        rho_kl = rho_kl_medium

    ws = bucket_df['Weighted_Sensitivity_WSk'].values

    # Sum of squares of weighted sensitivities
    sum_ws_sq = np.sum(ws**2)

    # Sum of cross-products
    cross_product_sum = 0
    if len(ws) > 1:
        # This loop calculates the sum of (rho_kl * WSk * WSl) for all pairs (k, l) where k != l
        for i in range(len(ws)):
            for j in range(len(ws)):
                if i != j:
                    cross_product_sum += rho_kl * ws[i] * ws[j]

    # The term inside the square root is floored at zero
    k_squared = max(sum_ws_sq + cross_product_sum, 0)

    return np.sqrt(k_squared)

print("\n### Step 4 & 5: Intra-Bucket Aggregation (Medium Scenario) ###")
print("Calculating bucket-specific capital (Kb) for the Medium Correlation Scenario.")

# Calculate Kb for each bucket
bucket_capital = {}
for bucket_id, group in net_sensitivities_df.groupby('Bucket'):
    bucket_capital[bucket_id] = calculate_kb(group, scenario='medium')

# Also calculate Sb for each bucket (sum of weighted sensitivities) for the next step
s_b_values = net_sensitivities_df.groupby('Bucket')['Weighted_Sensitivity_WSk'].sum().to_dict()

kb_results_df = pd.DataFrame({
    'Bucket': list(bucket_capital.keys()),
    'Kb_Medium': list(bucket_capital.values()),
    'Sb_Medium': list(s_b_values.values())
})

display(kb_results_df)



### Step 4 & 5: Intra-Bucket Aggregation (Medium Scenario) ###
Calculating bucket-specific capital (Kb) for the Medium Correlation Scenario.


,Bucket,Kb_Medium,Sb_Medium
0,6,142725.446896,143063.55
1,7,226026.495751,-226050.80


In [ ]:
# Cell 6: Step 6, 7 & 8 - Across-Bucket Aggregation (Medium Scenario)
# This function aggregates the bucket-level capital (Kb) to get the final
# risk class capital for a given scenario.

def calculate_final_capital(kb_df, scenario='medium'):
    """
    Calculates the final risk class capital based on Article 325f, paragraph 8.
    """
    gamma_medium = 0.20 # From Article 325au

    # Adjust correlation based on the scenario as per Article 325h
    if scenario == 'high':
        gamma = min(gamma_medium * 1.25, 1.0)
    elif scenario == 'low':
        gamma = max(2 * gamma_medium - 1, 0.75 * gamma_medium)
    else: # medium
        gamma = gamma_medium

    k_values = kb_df[f'Kb_{scenario.capitalize()}'].values
    s_values = kb_df[f'Sb_{scenario.capitalize()}'].values

    # Sum of squares of Kb values
    sum_k_sq = np.sum(k_values**2)

    # Sum of cross-bucket terms
    cross_bucket_sum = 0
    if len(k_values) > 1:
        for i in range(len(s_values)):
            for j in range(len(s_values)):
                if i != j:
                    cross_bucket_sum += gamma * s_values[i] * s_values[j]

    # The term inside the square root is floored at zero
    final_capital_sq = max(sum_k_sq + cross_bucket_sum, 0)

    return np.sqrt(final_capital_sq)

print("\n### Step 6, 7 & 8: Across-Bucket Aggregation (Medium Scenario) ###")
print("Aggregating bucket capital using cross-bucket correlations.")

# Calculate final capital for the medium scenario
medium_scenario_capital = calculate_final_capital(kb_results_df, scenario='medium')

print(f"\nMedium Scenario Commodity Delta Capital Requirement: {medium_scenario_capital:,.2f}")



### Step 6, 7 & 8: Across-Bucket Aggregation (Medium Scenario) ###
Aggregating bucket capital using cross-bucket correlations.

Medium Scenario Commodity Delta Capital Requirement: 241,914.61


In [ ]:
# Cell 7: Step 9 - Calculate High and Low Correlation Scenarios
# Per Article 325h, the calculation is repeated for high and low correlation scenarios.

print("\n### Step 9: Calculate High and Low Correlation Scenarios ###")

# --- High Correlation Scenario ---
bucket_capital_high = {}
for bucket_id, group in net_sensitivities_df.groupby('Bucket'):
    bucket_capital_high[bucket_id] = calculate_kb(group, scenario='high')
s_b_values_high = s_b_values # Sb values do not change with correlation scenario
kb_results_df['Kb_High'] = kb_results_df['Bucket'].map(bucket_capital_high)
kb_results_df['Sb_High'] = kb_results_df['Bucket'].map(s_b_values_high)
high_scenario_capital = calculate_final_capital(kb_results_df, scenario='high')

# --- Low Correlation Scenario ---
bucket_capital_low = {}
for bucket_id, group in net_sensitivities_df.groupby('Bucket'):
    bucket_capital_low[bucket_id] = calculate_kb(group, scenario='low')
s_b_values_low = s_b_values # Sb values do not change with correlation scenario
kb_results_df['Kb_Low'] = kb_results_df['Bucket'].map(bucket_capital_low)
kb_results_df['Sb_Low'] = kb_results_df['Bucket'].map(s_b_values_low)
low_scenario_capital = calculate_final_capital(kb_results_df, scenario='low')

# --- Display Summary ---
scenario_summary = pd.DataFrame([
    {'Scenario': 'Medium', 'Capital': medium_scenario_capital},
    {'Scenario': 'High', 'Capital': high_scenario_capital},
    {'Scenario': 'Low', 'Capital': low_scenario_capital}
])
display(scenario_summary.set_index('Scenario'))



### Step 9: Calculate High and Low Correlation Scenarios ###


,Capital
Scenario,
Medium,241914.608905
High,235364.246552
Low,248292.221884


In [ ]:
# Cell 8: Step 10 - Final Charge Calculation
# The final own funds requirement is the maximum of the three scenarios.

print("\n### Step 10: Final Charge Calculation ###")

final_capital_charge = scenario_summary['Capital'].max()

print(f"\nFinal Commodity Delta Capital Requirement: {final_capital_charge:,.2f}")

# Display the final summary card as a DataFrame for a clean output
summary_card = pd.DataFrame([{'Final Commodity Delta Capital Requirement': f"{final_capital_charge:,.2f}"}])
display(summary_card)


### Step 10: Final Charge Calculation ###

Final Commodity Delta Capital Requirement: 248,292.22


,Final Commodity Delta Capital Requirement
0,"248,292.22"
